# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [1]:
DEVICE_SOURCE = "Jing_device2"

TIME_WINDOW = "30s"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [3]:
sys_data_file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Jing_device2/merged_data.npz')
assert sys_data_file_path.exists(), 'Data not found'
vocab_file = Path("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Jing_device2/vocabulary.pkl")
assert vocab_file.exists(), f"Vocab file at {vocab_file} does not exist."

In [4]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [5]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [6]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
csv    530
zip     92
Name: filetype, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
FLS_data        91
KERN_data       91
RES_data        91
block_data      91
network_data    91
entropy_data    75
dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [7]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(63, 9)


### Get First TimeStamp

In [8]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

In [9]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-10-22 10:34:50 and 2023-10-31 18:45:29'

### Resample files

In [10]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [11]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

/tmp/ipykernel_98649/1828661645.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for source, group_df in df.groupby(["data_source"]):


Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (482, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (482, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
same_df shape: (4328, 57)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (482, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (483, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
same_df shape: (4332, 59)
Data type: DataSource.Resource

5

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [12]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

In [13]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['block:block_dirty_buffer', 'ext4:ext4_unlink_enter', 'ext4:ext4_writepages_result', 'ext4:ext4_es_lookup_extent_enter', 'block:block_touch_buffer', 'jbd2:jbd2_handle_stats', 'ext4:ext4_free_inode', 'seconds', 'jbd2:jbd2_handle_start', 'writeback:global_dirty_state', 'filemap:mm_filemap_add_to_page_cache', 'writeback:sb_clear_inode_writeback', 'block:block_bio_remap', 'block:block_unplug', 'jbd2:jbd2_start_commit', 'filemap:mm_filemap_delete_from_page_cache', 'jbd2:jbd2_checkpoint_stats', 'writeback:writeback_mark_inode_dirty', 'ext4:ext4_releasepage', 'writeback:writeback_dirty_inode_enqueue', 'ext4:ext4_journal_start', 'writeback:writeback_dirty_page', 'ext4:ext4_da_update_reserve_space', 'writeback:writeback_dirty_inode', 'block:block_bio_backmerge', 'block:block_getrq', 'label', 'device', 'data_source', 'kmem:kfree', 'fib:fib_table_lookup', 'tcp:tcp_probe', 'gpio:gpio_value', 'kmem:kmem_cache_alloc', 'timer:timer_start', 'kmem:mm_page_free', 'kmem:mm_page_alloc_zone_lo

In [14]:
new_dfs = []
for adf in df_mean_collection:
    print(adf.shape)
    adf = adf[~adf.index.duplicated(keep='first')]
    new_dfs.append(adf)

(4328, 54)
(4332, 56)
(4331, 73)
(4336, 3)
(4330, 2)


In [15]:
df_all = pd.concat(new_dfs, axis=1)
df_all.sample(2)

,ext4:ext4_truncate_enter,block:block_dirty_buffer_FLS_data,ext4:ext4_begin_ordered_truncate,ext4:ext4_unlink_enter_FLS_data,ext4:ext4_writepages_result_FLS_data,ext4:ext4_es_lookup_extent_enter_FLS_data,block:block_touch_buffer_FLS_data,ext4:ext4_da_reserve_space,jbd2:jbd2_handle_stats_FLS_data,block:block_rq_insert,...,timer:tick_stop,sched:sched_process_exec_RES_data,cpu-migrations_RES_data,branch-instructions,instructions,util,write_kbs,write_ops,(OLD) AverageLength,(OLD) VarianceLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-31 16:08:50,8.0,22.0,7.0,1.0,3.0,721.0,1992.0,6.0,1283.0,7.0,...,0.0,54.5,164.0,794859121.0,7.545901e+09,7.886667,3.8,30.933333,140.617021,47044.750939
2023-10-28 19:24:20,7.0,14.0,7.0,1.0,7.0,442.0,173.0,7.0,51.0,9.0,...,28.5,88.5,174.0,872566903.5,8.357634e+09,7.730000,1.4,11.595000,0.000000,0.000000


In [16]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [17]:
df_all.shape, len(SELECTED_FEATURES)

((21657, 189), 260)

In [18]:
df_all

,ext4:ext4_truncate_enter,block:block_dirty_buffer_FLS_data,ext4:ext4_begin_ordered_truncate,ext4:ext4_unlink_enter_FLS_data,ext4:ext4_writepages_result_FLS_data,ext4:ext4_es_lookup_extent_enter_FLS_data,block:block_touch_buffer_FLS_data,ext4:ext4_da_reserve_space,jbd2:jbd2_handle_stats_FLS_data,block:block_rq_insert,...,sched:sched_process_exec_RES_data,cpu-migrations_RES_data,branch-instructions,instructions,util,write_kbs,write_ops,(OLD) AverageLength,(OLD) VarianceLength,label
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,43.0,100.5,1.249920e+09,1.074348e+10,9.000,0.50,3.6,1136.121585,376891.379178,4_bashlite
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,43.0,100.5,1.249920e+09,1.074348e+10,9.000,0.50,3.6,1136.121585,376891.379178,4_bashlite
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,43.0,100.5,1.249920e+09,1.074348e+10,9.000,0.50,3.6,1136.121585,376891.379178,4_bashlite
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,43.0,100.5,1.249920e+09,1.074348e+10,9.000,0.50,3.6,1136.121585,376891.379178,4_bashlite
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,43.0,100.5,1.249920e+09,1.074348e+10,9.000,0.50,3.6,1136.121585,376891.379178,4_bashlite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31 18:44:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.0,122.0,7.677031e+08,7.520056e+09,8.895,3.45,26.2,937.304348,492739.060870,8_bdvl
2023-10-31 18:44:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.0,122.0,7.677031e+08,7.520056e+09,8.895,3.45,26.2,937.304348,492739.060870,8_bdvl
2023-10-31 18:44:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.0,122.0,7.677031e+08,7.520056e+09,8.895,3.45,26.2,937.304348,492739.060870,8_bdvl
2023-10-31 18:44:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.0,122.0,7.677031e+08,7.520056e+09,8.895,3.45,26.2,937.304348,492739.060870,8_bdvl


# Add Sys Call Data

In [19]:
def read_in_preprocessed_Xi_sensor_2():
    with np.load(sys_data_file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer(vocab_file=vocab_file)
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Xi_sensor_2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(10625, 148)
Loaded set: ('accept', 'accept4', 'access', 'add_key', 'adjtimex', 'bind', 'brk', 'capget', 'capset', 'chdir', 'chmod', 'chown32', 'clock_gettime', 'clone', 'close', 'connect', 'dup', 'dup2', 'dup3', 'epoll_ctl', 'epoll_wait', 'eventfd2', 'execve', 'exit', 'exit_group', 'faccessat', 'fadvise64_64', 'fallocate', 'fchmod', 'fchown32', 'fcntl64', 'flistxattr', 'flock', 'fstat64', 'fstatat64', 'fstatfs64', 'fsync', 'ftruncate64', 'futex', 'getcwd', 'getdents64', 'getegid32', 'geteuid32', 'getgid32', 'getgroups32', 'getpeername', 'getpgid', 'getpgrp', 'getpid', 'getppid', 'getrandom', 'getresgid32', 'getresuid32', 'getsid', 'getsockname', 'getsockopt', 'gettid', 'getuid32', 'getxattr', 'inotify_add_watch', 'inotify_rm_watch', 'ioctl', 'ioprio_set', 'kcmp', 'keyctl', 'kill', 'lchown32', 'lgetxattr', 'llseek', 'lstat64', 'madvise', 'mkdir', 'mknod', 'mmap2', 'mount', 'mprotect', 'mremap', 'msync', 'munmap', 'name_to_handle_at', 'newselect', 'openat', 'pause', 'perf_even

In [20]:
sys_df.head(2)

,accept,accept4,access,add_key,adjtimex,bind,brk,capget,capset,chdir,...,umount2,uname,unlink,unlinkat,utimensat,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-22 10:34:50,0.0,0.0,0.001177,0.0,0.000135,0.000016,0.001693,0.000069,0.000025,0.000022,...,0.0,0.000423,0.000012,0.0,0.0,0.000640,0.000011,0.011494,0.000009,4_bashlite
2023-10-22 10:35:02,0.0,0.0,0.000574,0.0,0.000040,0.000005,0.000568,0.000000,0.000000,0.000000,...,0.0,0.000150,0.000004,0.0,0.0,0.000338,0.000017,0.018922,0.000000,4_bashlite


In [21]:
if sys_df["label"].unique().shape[0] != 9:
    raise Exception(f'Not all labels!!!: {sys_df["label"].unique()}')
sys_df["label"].unique()

array(['4_bashlite', '5_httpbackdoor', '7_backdoor', '3_thetick',
       '6_beurk', '1_normal', '2_ransomware', '9_xmrig', '8_bdvl'],
      dtype=object)

In [22]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [23]:
sys_df_mean.head(2)

,chdir,execve,setxattr,sendto,getpgrp,epoll_wait,recvfrom,sendmsg,getegid32,setuid32,...,getppid,geteuid32,setresuid32,timerfd_settime,setitimer,dup2,statfs64,sigaltstack,getrandom,getpid
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-22 10:34:50,0.000011,0.000263,0.000009,0.000034,0.0,0.000456,0.000013,0.000806,0.000510,0.000004,...,0.000016,0.000545,0.000022,0.000076,0.000006,0.000346,0.000034,0.000016,0.000076,0.007339
2023-10-22 10:35:20,0.000002,0.000226,0.000006,0.000041,0.0,0.000503,0.000016,0.001372,0.000465,0.000000,...,0.000009,0.000492,0.000005,0.000100,0.000006,0.000256,0.000021,0.000017,0.000182,0.011215


In [24]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (44223, 286)


In [25]:
all_df1.head(4)

,ext4:ext4_truncate_enter,block:block_dirty_buffer_FLS_data,ext4:ext4_begin_ordered_truncate,ext4:ext4_unlink_enter_FLS_data,ext4:ext4_writepages_result_FLS_data,ext4:ext4_es_lookup_extent_enter_FLS_data,block:block_touch_buffer_FLS_data,ext4:ext4_da_reserve_space,jbd2:jbd2_handle_stats_FLS_data,block:block_rq_insert,...,getppid,geteuid32,setresuid32,timerfd_settime,setitimer,dup2,statfs64,sigaltstack,getrandom,getpid
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,0.000016,0.000545,0.000022,0.000076,0.000006,0.000346,0.000034,0.000016,0.000076,0.007339
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,0.000016,0.000545,0.000022,0.000076,0.000006,0.000346,0.000034,0.000016,0.000076,0.007339
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,0.000016,0.000545,0.000022,0.000076,0.000006,0.000346,0.000034,0.000016,0.000076,0.007339
2023-10-22 10:34:50,1.0,6.0,1.0,0.0,2.0,2976.5,3910.0,1.0,7.0,582.0,...,0.000016,0.000545,0.000022,0.000076,0.000006,0.000346,0.000034,0.000016,0.000076,0.007339


In [26]:
spath = repo_base_path / "training" / "data" / "all_30s" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))